In [9]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime
from config import password, login, server
import time

In [2]:
mt5.initialize(login=login, server=server, password=password)

True

In [76]:
# settings
symbol = 'USDJPY'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 1000

fsma_period = 14
ssma_period = 28

In [77]:
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()

df.dropna(inplace=True)
df

,time,open,close,low,high,slow_sma,fast_sma
27,2023-10-03 15:45:00,149.879,149.893,149.874,149.933,149.929429,149.913857
28,2023-10-03 15:50:00,149.889,149.884,149.866,149.895,149.927321,149.910000
29,2023-10-03 15:55:00,149.881,149.881,149.853,149.894,149.924857,149.906071
30,2023-10-03 16:00:00,149.887,150.048,149.871,150.064,149.928750,149.914357
31,2023-10-03 16:05:00,150.049,150.083,150.029,150.154,149.933071,149.926429
...,...,...,...,...,...,...,...
995,2023-10-09 00:25:00,149.180,149.139,149.139,149.206,149.176179,149.109714
996,2023-10-09 00:30:00,149.140,149.151,149.138,149.196,149.170071,149.113786
997,2023-10-09 00:35:00,149.147,149.156,149.125,149.178,149.163714,149.118714
998,2023-10-09 00:40:00,149.157,149.222,149.154,149.228,149.160107,149.126500
